In [ ]:
import pandas as pd
import numpy as np
import requests
import zipfile
import io
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Charger le jeu de données
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
response = requests.get(url)
with zipfile.ZipFile(io.BytesIO(response.content)) as z:
    with z.open('SMSSpamCollection') as f:
        df = pd.read_csv(f, sep='\t', names=['label', 'message'])

# Prétraitement des données
X = df['message']
y = df['label'].map({'ham': 0, 'spam': 1})  # Convertir les étiquettes en nombres

# Diviser le jeu de données
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convertir les messages en vecteurs
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Initialiser les modèles
models = {
    'Naïf Bayes': MultinomialNB(),
    'SVM': SVC(),
    'Arbre de Décision': DecisionTreeClassifier()
}

# Entraîner et évaluer chaque modèle
accuracies = {}
execution_times = {}

for name, model in models.items():
    # Mesurer le temps d'exécution pour l'entraînement
    start_time = time.time()
    model.fit(X_train_vectorized, y_train)
    training_time = time.time() - start_time

    # Mesurer le temps d'exécution pour la prédiction
    start_time = time.time()
    y_pred = model.predict(X_test_vectorized)
    prediction_time = time.time() - start_time

    # Calculer l'exactitude
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[name] = accuracy
    execution_times[name] = (training_time, prediction_time)

# Afficher les résultats
for name, accuracy in accuracies.items():
    training_time, prediction_time = execution_times[name]
    print(f'Exactitude du classificateur {name} : {accuracy * 100:.2f}%')
    print(f'Temps d\'entraînement : {training_time:.4f} secondes')
    print(f'Temps de prédiction : {prediction_time:.4f} secondes')
    print('---')

# Identifier le meilleur modèle
best_model = max(accuracies, key=accuracies.get)
print(f'\nLe meilleur modèle est : {best_model} avec une exactitude de {accuracies[best_model] * 100:.2f}%')


Exactitude du classificateur Naïf Bayes : 99.19%
Temps d'entraînement : 0.0037 secondes
Temps de prédiction : 0.0005 secondes
---
Exactitude du classificateur SVM : 98.65%
Temps d'entraînement : 1.1895 secondes
Temps de prédiction : 0.2802 secondes
---
Exactitude du classificateur Arbre de Décision : 97.04%
Temps d'entraînement : 0.1786 secondes
Temps de prédiction : 0.0009 secondes
---

Le meilleur modèle est : Naïf Bayes avec une exactitude de 99.19%
